In [3]:
# Install geopandas: Used for reading and manipulating spatial data. 
# geopandas is used to load the GeoJSON file containing ward boundaries and perform spatial operations.
!pip install geopandas

# Install shapely: Used for manipulation and analysis of planar geometric objects.  
# Shapely is used to create Point objects representing geographic locations (latitude and longitude) 
# and to check if these points fall within the ward boundaries defined in the GeoJSON file.
!pip install shapely

# Install pandas: Used for data manipulation and analysis. 
# pandas is used to read the CSV file containing river water data, manipulate this data by adding a new column for ward information, 
# and then save the modified data back to a new CSV file.
!pip install pandas


In [2]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd

geoJsonFile = '../../initial-csv-data/Wards-pre-2013.json'
river_water_csv = '../../working-csv-data/03-river-data-with-lat-lng.csv'
river_water_with_ward = '../../working-csv-data/04-river-data-with-ward.csv'

# Load the GeoJSON file
geo_df = gpd.read_file(geoJsonFile)

def find_ward(lat, lon):
    """
    Returns the ward name for the given latitude and longitude.

    Parameters:
    - lat (float): Latitude of the point.
    - lon (float): Longitude of the point.

    Returns:
    - str: Name of the ward containing the point, or 'Not found' if the point is not within any ward.
    """
    point = Point(lon, lat)  # Note: Point takes (longitude, latitude)
    for index, row in geo_df.iterrows():
        if row['geometry'].contains(point):
            return row['WARDS']
    return 'Not found'

# Load the river water data file

data_df = pd.read_csv(river_water_csv)

# Ensure latitude and longitude columns are present
if 'latitude' in data_df.columns and 'longitude' in data_df.columns:
    # Use the find_ward function to determine the ward name and add it to a new column
    data_df['Ward'] = data_df.apply(lambda row: find_ward(row['latitude'], row['longitude']), axis=1)
else:
    result = "The CSV file does not have the required 'latitude' and 'longitude' columns."

# Save the updated DataFrame to a new CSV file
data_df.to_csv(river_water_with_ward, index=False)